##### Copyright 2018 The TensorFlow Authors.

# Heart Disease prediction

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
#from tensorflow import tfx
from tensorflow.keras import layers

print(tf.__version__)

1.13.1


### Get the data
First download the dataset.

Import it using pandas

In [2]:
# column_names = ['age','sex','chest_pain_experienced','resting_blood_pressure','cholesterol','fasting_blood_sugar','Resting_electrocardiographic_measurement',
#                   'maximum_heart_rate','exercise_induced_angina','ST_depression','peak_ST_slope','major_vessels_number','thalassemia',
#                    'target']
raw_dataset = pd.read_csv('heart.csv',
                      na_values = "?", comment='\t',
                       skipinitialspace=True)

dataset = raw_dataset.copy()
dataset.tail()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0
302,57,0,1,130,236,0,0,174,0,0.0,1,1,2,0


### Clean the data

The dataset contains a few unknown values. 

In [3]:
dataset.isna().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

To keep this initial tutorial simple drop those rows. 

In [4]:
dataset = dataset.dropna()

The `"Origin"` column is really categorical, not numeric. So convert that to a one-hot:

### Split the data into train and test

Now split the dataset into a training set and a test set.

We will use the test set in the final evaluation of our model.

In [0]:
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)
print(train_dataset[:-1].tail())


### Inspect the data

Have a quick look at the joint distribution of a few pairs of columns from the training set.

In [0]:
#sns.set(style="ticks", color_codes=True)
sns.pairplot(train_dataset, diag_kind="kde")

Also look at the overall statistics:

In [0]:
train_stats = train_dataset.describe()
train_stats.pop("target")
train_stats = train_stats.transpose()
train_stats

### Split features from labels

Separate the target value, or "label", from the features. This label is the value that you will train the model to predict.

In [0]:
train_labels = train_dataset.pop('target')
test_labels = test_dataset.pop('target')

### Normalize the data

Look again at the `train_stats` block above and note how different the ranges of each feature are.

It is good practice to normalize features that use different scales and ranges. Although the model *might* converge without feature normalization, it makes training more difficult, and it makes the resulting model dependent on the choice of units used in the input. 

Note: Although we intentionally generate these statistics from only the training dataset, these statistics will also be used to normalize the test dataset. We need to do that to project the test dataset into the same distribution that the model has been trained on.

In [0]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)



This normalized data is what we will use to train the model.

Caution: The statistics used to normalize the inputs here (mean and standard deviation) need to be applied to any other data that is fed to the model, along with the one-hot encoding that we did earlier.  That includes the test set as well as live data when the model is used in production.

## The model

### Build the model

Let's build our model. Here, we'll use a `Sequential` model with two densely connected hidden layers, and an output layer that returns a single, continuous value. The model building steps are wrapped in a function, `build_model`, since we'll create a second model, later on.

In [0]:
# def build_model():
#   model = keras.Sequential([
#     layers.Dense(13, activation=tf.nn.relu, input_shape=[len(train_dataset.keys())]),
#     layers.Dense(11, activation=tf.nn.relu),
#     layers.Dense(1,activation=tf.nn.sigmoid)
#   ])

#   optimizer = tf.keras.optimizers.RMSprop(0.001)

#   model.compile(loss='binary_crossentropy',
#                 optimizer=optimizer,
#                 metrics=['accuracy','binary_crossentropy'])
#   return model

L2 Regularization applied to model

In [0]:
# def build_model():
#   model = keras.Sequential([
#     layers.Dense(13, kernel_regularizer=keras.regularizers.l2(0.001), activation=tf.nn.relu, input_shape=[len(train_dataset.keys())]),
#     layers.Dense(11, kernel_regularizer=keras.regularizers.l2(0.001), activation=tf.nn.relu),
#     layers.Dense(1,activation=tf.nn.sigmoid)
#   ])

#   optimizer = tf.keras.optimizers.RMSprop(0.001)

#   model.compile(loss='binary_crossentropy',
#                 optimizer=optimizer,
#                 metrics=['accuracy','binary_crossentropy'])
#   return model

Dropout layer added tothe model

In [0]:
def build_model():
  model = keras.Sequential([
    layers.Dense(13, activation=tf.nn.relu, input_shape=[len(train_dataset.keys())]),
       keras.layers.Dropout(0.7),
    layers.Dense(11, activation=tf.nn.relu),
       keras.layers.Dropout(0.7),
    layers.Dense(1,activation=tf.nn.sigmoid)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='binary_crossentropy',
                optimizer=optimizer,
                metrics=['accuracy','binary_crossentropy'])
  return model

In [0]:
model = build_model()

### Inspect the model

Use the `.summary` method to print a simple description of the model

In [0]:
model.summary()


Now try out the model. Take a batch of `10` examples from the training data and call `model.predict` on it.

In [0]:
example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
example_result

### Train the model

Train the model for 1000 epochs, and record the training and validation accuracy in the `history` object.

In [0]:
# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 1000

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])

Visualize the model's training progress using the stats stored in the `history` object.

In [0]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [0]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch
  
  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.plot(hist['epoch'], hist['acc'],
           label='Train Accuracy')
  plt.plot(hist['epoch'], hist['val_acc'],
           label = 'Val Accuracy')
  plt.ylim([0,5])
  plt.legend()
  
  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.plot(hist['epoch'], hist['loss'],
           label='Train Loss')
  plt.plot(hist['epoch'], hist['val_loss'],
           label = 'Val Loss')
  plt.ylim([0,20])
  plt.legend()
  plt.show()


plot_history(history)

In [0]:
# def plot_history(history):
#   hist = pd.DataFrame(history.history)
#   hist['epoch'] = history.epoch
  
#   plt.figure()
#   plt.xlabel('Epoch')
#   plt.ylabel('Mean Abs Error ')
#   plt.plot(hist['epoch'], hist['mean_absolute_error'],
#            label='Train Error')
#   plt.plot(hist['epoch'], hist['val_mean_absolute_error'],
#            label = 'Val Error')
#   plt.ylim([0,5])
#   plt.legend()
  
#   plt.figure()
#   plt.xlabel('Epoch')
#   plt.ylabel('Mean Square Error ')
#   plt.plot(hist['epoch'], hist['mean_squared_error'],
#            label='Train Error')
#   plt.plot(hist['epoch'], hist['val_mean_squared_error'],
#            label = 'Val Error')
#   plt.ylim([0,20])
#   plt.legend()
#   plt.show()


# plot_history(history)

In [0]:
model = build_model()

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(normed_train_data, train_labels, epochs=EPOCHS,
                    validation_split = 0.2, verbose=0, callbacks=[early_stop, PrintDot()])

plot_history(history)

In [0]:
loss,acc,binc = model.evaluate(normed_test_data, test_labels, verbose=1)
# print(loss)
print("Testing set Mean Abs Error: {:5.2f} ".format(loss))

### Make predictions

Finally, predict MPG values using data in the testing set:

In [0]:
test_predictions = model.predict(normed_test_data).flatten()

plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values ')
plt.ylabel('Predictions ')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
# _ = plt.plot([-100, 100], [-100, 100])


It looks like our model predicts reasonably well. Let's take a look at the error distribution.

In [0]:
error = test_predictions - test_labels
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error ")
_ = plt.ylabel("Count")

It's not quite gaussian, but we might expect that because the number of samples is very small.